<a href="https://colab.research.google.com/github/pradeepvaranasi/RideSharing_riderAttrition_casestudy/blob/main/RideSharing_Casestudy_EnsembleLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Problem Statement

Recruiting and retaining drivers is seen by industry watchers as a tough battle for Ola. Churn among drivers is high and it’s very easy for drivers to stop working for the service on the fly or jump to Uber depending on the rates.

As the companies get bigger, the high churn could become a bigger problem. To find new drivers, Ola is casting a wide net, including people who don’t have cars for jobs. But this acquisition is really costly. Losing drivers frequently impacts the morale of the organization and acquiring new drivers is more expensive than retaining existing ones.

You are working as a data scientist with the Analytics Department of Ola, focused on driver team attrition. You are provided with the monthly information for a segment of drivers for 2019 and 2020 and tasked to predict whether a driver will be leaving the company or not based on their attributes like

In [2]:
# Imporing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

In [3]:
# Establishing the connection between Colab and Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Importing the data from gdrive
data = pd.read_csv('drive/MyDrive/ola_driver_scaler.csv', index_col=False)
data = data.iloc[:,2:]
data.head(2)

,Driver_ID,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
0,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,2381060,2
1,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,-665480,2


Removing the first two columns index (not relevant) and MMM-YY (98% missing values)

In [5]:
df = data.copy()
df.head(2)

,Driver_ID,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
0,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,2381060,2
1,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,-665480,2


In [6]:
df.shape

(19104, 12)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19104 entries, 0 to 19103
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Driver_ID             19104 non-null  int64  
 1   Age                   19043 non-null  float64
 2   Gender                19052 non-null  float64
 3   City                  19104 non-null  object 
 4   Education_Level       19104 non-null  int64  
 5   Income                19104 non-null  int64  
 6   Dateofjoining         19104 non-null  object 
 7   LastWorkingDate       1616 non-null   object 
 8   Joining Designation   19104 non-null  int64  
 9   Grade                 19104 non-null  int64  
 10  Total Business Value  19104 non-null  int64  
 11  Quarterly Rating      19104 non-null  int64  
dtypes: float64(2), int64(7), object(3)
memory usage: 1.7+ MB


* Missing values found in the following columns:
    1. Age
    2. Gender
    3. LastWorkingDate (Attrition - derive target variable)

* Datatype conversion required for Date Columns


#### Data Preprocessing and Feature Engineering

The data needs to be aggregated at driver_id level as there are multiple records for each driver on monthly basis

In [8]:
df.head()

,Driver_ID,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
0,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,2381060,2
1,1,28.0,0.0,C23,2,57387,24/12/18,NaN,1,1,-665480,2
2,1,28.0,0.0,C23,2,57387,24/12/18,03/11/19,1,1,0,2
3,2,31.0,0.0,C7,2,67016,11/06/20,NaN,2,2,0,1
4,2,31.0,0.0,C7,2,67016,11/06/20,NaN,2,2,0,1


In [9]:
# Convering the date like features to date datatype

df['Dateofjoining'] = pd.to_datetime(df['Dateofjoining'])
df['LastWorkingDate'] = pd.to_datetime(df['LastWorkingDate'])

In [10]:
df.head(5)

,Driver_ID,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating
0,1,28.0,0.0,C23,2,57387,2018-12-24,NaT,1,1,2381060,2
1,1,28.0,0.0,C23,2,57387,2018-12-24,NaT,1,1,-665480,2
2,1,28.0,0.0,C23,2,57387,2018-12-24,2019-03-11,1,1,0,2
3,2,31.0,0.0,C7,2,67016,2020-11-06,NaT,2,2,0,1
4,2,31.0,0.0,C7,2,67016,2020-11-06,NaT,2,2,0,1


In [11]:
# CChecking info for datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19104 entries, 0 to 19103
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Driver_ID             19104 non-null  int64         
 1   Age                   19043 non-null  float64       
 2   Gender                19052 non-null  float64       
 3   City                  19104 non-null  object        
 4   Education_Level       19104 non-null  int64         
 5   Income                19104 non-null  int64         
 6   Dateofjoining         19104 non-null  datetime64[ns]
 7   LastWorkingDate       1616 non-null   datetime64[ns]
 8   Joining Designation   19104 non-null  int64         
 9   Grade                 19104 non-null  int64         
 10  Total Business Value  19104 non-null  int64         
 11  Quarterly Rating      19104 non-null  int64         
dtypes: datetime64[ns](2), float64(2), int64(7), object(1)
memory usage: 1.7+ M

In [12]:
# KNN missing value imputation
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=3)

In [13]:
columns_to_impute = ['Age', 'Gender']
df_num = df[columns_to_impute]

In [14]:
# Fit and transform only the selected num columns
df_num_imputed = pd.DataFrame(imputer.fit_transform(df_num), columns=columns_to_impute)

In [15]:
# Update the original DataFrame with the imputed values
df[columns_to_impute] = df_num_imputed

In [ ]:
df.head()

In [17]:
# Converting gender from float to int datatype
df['Age'] = df['Age'].astype(int)
df['Gender'] = df['Gender'].astype(int)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19104 entries, 0 to 19103
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Driver_ID             19104 non-null  int64         
 1   Age                   19104 non-null  int64         
 2   Gender                19104 non-null  int64         
 3   City                  19104 non-null  object        
 4   Education_Level       19104 non-null  int64         
 5   Income                19104 non-null  int64         
 6   Dateofjoining         19104 non-null  datetime64[ns]
 7   LastWorkingDate       1616 non-null   datetime64[ns]
 8   Joining Designation   19104 non-null  int64         
 9   Grade                 19104 non-null  int64         
 10  Total Business Value  19104 non-null  int64         
 11  Quarterly Rating      19104 non-null  int64         
dtypes: datetime64[ns](2), int64(9), object(1)
memory usage: 1.7+ MB


In [37]:
# Remove duplicate occurrences of the same driver data
unique_drivers_df = df.drop_duplicates(subset='Driver_ID')

# Group by Driver_ID and aggregate features
grouped_df = unique_drivers_df.groupby('Driver_ID').agg({
    'Age': 'first',
    'Gender': 'first',
    'City': 'first',
    'Education_Level': 'first',
    'Income': 'mean',
    'Dateofjoining': 'first',
    'LastWorkingDate': 'last',
    'Joining Designation': 'first',
    'Grade': 'first',
    'Total Business Value': 'sum',
    'Quarterly Rating': 'last'
})


In [38]:
# Feature Engineering
grouped_df['Rating_Increase'] = grouped_df['Quarterly Rating'].diff().gt(0).astype(int)
grouped_df['Income_Increase'] = grouped_df['Income'].diff().gt(0).astype(int)

In [39]:
# Target variable creation
grouped_df['Target'] = grouped_df['LastWorkingDate'].notnull().astype(int)

In [41]:
# Display the final DataFrame
grouped_df.head()

,Age,Gender,City,Education_Level,Income,Dateofjoining,LastWorkingDate,Joining Designation,Grade,Total Business Value,Quarterly Rating,Rating_Increase,Income_Increase,Target
Driver_ID,,,,,,,,,,,,,,
1,28,0,C23,2,57387.0,2018-12-24,NaT,1,1,2381060,2,0,0,0
2,31,0,C7,2,67016.0,2020-11-06,NaT,2,2,0,1,0,1,0
4,43,0,C13,2,65603.0,2019-12-07,NaT,2,2,0,1,0,0,0
5,29,0,C9,0,46368.0,2019-01-09,NaT,1,1,0,1,0,0,0
6,31,1,C11,1,78728.0,2020-07-31,NaT,3,3,0,1,0,1,0


In [42]:
grouped_df.shape

(2381, 14)